In [ ]:
import panel as pn
import param

pn.extension()

The ``panel.Param`` pane allows customizing the widgets, layout and style of the parameters of a `param.Parametrized` Class.

#### Parameters:

The basic parameters are

* **`object`** (param.parameterized.Parameters): The `param` attribute of a `param.Parameterized` Class
* **`parameters`** (None or List[str]): A list identifying the subset of parameters to include in the Pane.
* **`widgets`** (Dict): A Dictionary specifying which parameters and widgets to use for a given parameter. You can also specify widget attributes.

The more advanced parameters give you more control and they are
 
* **`default_layout`** (ClassSelector) A layout like Column, Row etc or a Custom GridBox.
* **`expand`** (bool) Whether or not to expand
* **`expand_button`** (None or Button) The expand button
* **`expand_layout`** (ClassSelector) A layout like Column, Row etc. or a Custom GridBox.
* **`show_labels`** (bool) Whether or not to show labels
* **`show_name`** (bool) Whether or not to show the name of the Parameterized Class 

For more layout and styling related parameters see the [customization user guide](../../user_guide/Customization.ipynb).

For an alternative example of using `panel.Param` see the [parameters user guide](../../user_guide/Param.ipynb).

___

Lets build a model of a cycling Athlete and his PowerCurve. 

The PowerCurve is a recording of his maximum power output in Watt per kg for fixed durations of time.

In [ ]:
import datetime
import pandas as pd
import hvplot.pandas

DATE_BOUNDS = (datetime.date(1900, 1, 1), datetime.datetime.now().date())

class PowerCurve(param.Parameterized):
    ten_sec = param.Number(1079)
    ten_sec_date = param.Date(datetime.date(2018, 8, 21), bounds=DATE_BOUNDS)
    one_min = param.Number(684)
    one_min_date = param.Date(datetime.date(2017, 8, 31), bounds=DATE_BOUNDS)
    ten_min = param.Number(419)
    ten_min_date = param.Date(datetime.date(2017, 9, 22), bounds=DATE_BOUNDS)
    twenty_min = param.Number(398)
    twenty_min_date = param.Date(datetime.date(2017, 9, 22), bounds=DATE_BOUNDS)
    one_hour = param.Number(319)
    one_hour_date = param.Date(datetime.date(2017, 8, 6), bounds=DATE_BOUNDS)
    
    @param.depends("ten_sec", "one_min", "ten_min", "twenty_min", "one_hour")
    def plot(self):
        data = {
            "duration": [10 / 60, 1, 10, 20, 60],
            "power": [self.ten_sec, self.one_min, self.ten_min, self.twenty_min, self.one_hour],
        }
        dataframe = pd.DataFrame(data)
        line_plot = dataframe.hvplot.line(
            x="duration", y="power", width=300, line_color="#007BFF", line_width=3,
        )
        scatter_plot = dataframe.hvplot.scatter(x="duration", y="power", width=300).opts(
            marker="o", size=6, color="#007BFF"
        )
        fig = line_plot * scatter_plot
        gridstyle = {"grid_line_color": "black", "grid_line_width": 0.1}
        fig = fig.opts(
            responsive=True,
            toolbar=None,
            yticks=list(range(0, 1600, 200)),
            ylim=(0, 1500),
            gridstyle=gridstyle,
            show_grid=True,
        )
        return fig
        
        

class Athlete(param.Parameterized):
    name_ = param.String("P.A. Nelson")
    birthday = param.Date(datetime.date(1976, 9, 17), bounds=DATE_BOUNDS)
    weight = param.Number(default=82, bounds=(20,300))
    power_curve = param.ClassSelector(class_=PowerCurve, default=PowerCurve())
    
athlete = Athlete()

The `pn.Param` can be used to view and edit the models.

Try clicking the `...` button. This will expand the PowerCurve if running in an interactive notebook.

In [ ]:
pn.Param(athlete)

The default Name and Birthday widgets are slow to use. So lets change them to a DatePicker and a LiteralInput for ints.

In [ ]:
pn.Param(athlete, widgets={"birthday": pn.widgets.DatePicker, "weight": pn.widgets.LiteralInput})

Lets expand the power curve by default.

In [ ]:
pn.Param(
    athlete, 
    widgets={
        "birthday": pn.widgets.DatePicker, 
        "weight": pn.widgets.LiteralInput
    }, expand=True)

Now lets try to display the Name and Birthday only and in a Row.

In [ ]:
pn.Param(
    athlete,
    widgets={"birthday": pn.widgets.DatePicker},
    parameters=["name_", "birthday"],
    show_name=False,
    default_layout=pn.Row,
    width=400
)

Lets customize the view of the Athlete some more

In [ ]:
athlete_view = pn.Param(
    athlete,
    widgets={
        "birthday": pn.widgets.DatePicker, 
        "weight": {"type": pn.widgets.LiteralInput, "width": 100}
    },
    parameters=["name_", "birthday", "weight"],
    show_name=False,
    default_layout=pn.Row,
    width=600
)
athlete_view

Lets take a look at the PowerCurve

In [ ]:
pn.Param(athlete.power_curve)

The PowerCurve layout is not that tidy. Lets change the layout to two columns.

In [ ]:
class GridBoxWithTwoColumns(pn.GridBox):
    def __init__(self, *objects, **params):
        super().__init__(*objects, **params, ncols=2)
        
power_curve_columns_view = pn.Param(
    athlete.power_curve, 
    default_layout=GridBoxWithTwoColumns, 
    show_name=False,
    widgets = {
        "ten_sec_date": pn.widgets.DatePicker, 
        "one_min_date": pn.widgets.DatePicker, 
        "ten_min_date": pn.widgets.DatePicker, 
        "twenty_min_date": pn.widgets.DatePicker, 
        "one_hour_date": pn.widgets.DatePicker, 
    }
)
power_curve_columns_view



Lets put a plot of the PowerCurve in the mix

In [ ]:
power_curve_view = pn.Row(
    power_curve_columns_view,
    pn.layout.VSpacer(width=50),
    athlete.power_curve.plot,
)
power_curve_view

And finally. Lets put the things together

In [ ]:
pn.Column(
    pn.pane.Markdown("### Athlete"), 
    athlete_view, 
    pn.pane.Markdown("#### Power Curve"), 
    power_curve_view,
)